In [1]:
import advent
import numpy as np
data = [int(i) for i in advent.get_lines(16)[0]]
data[:10]

[5, 9, 7, 9, 3, 5, 1, 3, 5, 1]

In [2]:
last_digit = lambda d: d % 10 if d >= 0 else (-d) % 10

def apply_pattern(input, pattern):
    # input is list of length N
    # pattern is list of length M (already repeated if needed)
    # extend pattern by calling np.tile on it so it's longer than input,
    #   then clipping to same length as input, and skipping the very first element of the pattern
    # then return dot product
    tile = (len(input) // len(pattern)) + 1  # +1 so we can always skip the first element
    pattern_extended = np.tile(pattern, tile)[1:len(input)+1]
    res = np.dot(input, pattern_extended)
    return last_digit(res)

In [3]:
def apply_full_pattern(input, base_pattern=np.array([0, 1, 0, -1])):
    output = np.zeros(len(input))
    for ix, _ in enumerate(input):
        pattern = np.repeat(base_pattern, ix+1)
        output[ix] = apply_pattern(input, pattern)
    return output

test_input = np.array([1, 2, 3, 4, 5, 6, 7, 8])
apply_full_pattern(apply_full_pattern(test_input))

array([3., 4., 0., 4., 0., 4., 3., 8.])

In [ ]:
def apply_full_pattern_n_times(input, n=100, base_pattern=np.array([0, 1, 0, -1])):
    for _ in range(n):
        input = apply_full_pattern(input, base_pattern)
    return input

test_input = np.array([int(i) for i in '80871224585914546619083218645595'])
''.join(str(int(i)) for i in apply_full_pattern_n_times(test_input)) # should start with 24176176

array([2., 4., 1., 7., 6., 1., 7., 6., 4., 8., 0., 9., 1., 9., 0., 4., 6.,
       1., 1., 4., 0., 3., 8., 7., 6., 3., 1., 9., 5., 5., 9., 5.])

In [9]:
''.join(str(int(i)) for i in apply_full_pattern_n_times(data, n=100)[:10])

'1687560704'